In [1]:
import polars as pl
import pandas as pd
import polars.datatypes as T
import numpy as np

from datetime import datetime, time
import sys
sys.path.insert(0, 'C:\\Users\\rockh\\Repositories\\Tsuro\\')

from tsuro.datasets import load_cba_trades
from tsuro.preprocessing import cast_strings_to_datetime

C:\Users\rockh\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from sklearn.utils.parallel import Parallel, delayed

In [5]:
from sklearn.utils.random import sample_without_replacement

def _generate_indices(
    n_population: int,
    n_samples: int,
    bootstrap: bool = False,
    random_state = np.random.RandomState(),
) -> np.array:
    """
    Generate random indices
    """

    if bootstrap:
        indices = random_state.randint(0, n_population, n_samples)
    else:
        indices = sample_without_replacement(
            n_population, n_samples, random_state=random_state
        )
    return indices

In [10]:
n_samples = 500
n_indices = 1000

def filter_overlap_matrix(
    n_samples: int,
    n_indices: int,
    overlap_matrix
) -> np.ndarray:
    """
    Filter overlap_matrix
    """
    if n_samples is None:
        return overlap_matrix

    if n_samples < n_indices:
        overlap_matrix_indices = _generate_indices(n_indices, n_samples)
        overlap_matrix = overlap_matrix[:, overlap_matrix_indices]
    elif n_samples > n_indices:
        raise ValueError(
            f"X.shape[0] ({n_samples}) exceeds number of available indices in overlap matrix, overlap_matrix.shape[1] ({n_indices})."
        )
    else:
        overlap_matrix = overlap_matrix

    return overlap_matrix



In [11]:
new_overlap_matrix = filter_overlap_matrix(
    n_samples,
    n_indices,
    overlap_matrix_bars_np
)
new_overlap_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [3]:
# import math
# import time
# from joblib import Parallel, delayed

# t1 = time.time()
# results = [math.factorial(x) for x in range(10000)]
# t2 = time.time()
# print(t2-t1)

# t1 = time.time()
# results = Parallel(n_jobs = 2)(delayed(math.factorial)(x) for x in range(10000))
# t2 = time.time()
# print(t2-t1)

In [4]:
# t1 = time.time()
# results = Parallel(n_jobs = 6)((delayed(math.factorial)(x) for x in range(10000)))
# t2 = time.time()

# print(t2-t1)

In [8]:
# Overlap Matrix
overlap_matrix = pl.read_csv("overlap_matrix_test.csv")

overlap_matrix_bars = overlap_matrix.with_columns(
    pl.when(
        pl.col("time_start") == pl.col("time_end"),
    ).then(1).otherwise(0).alias("zero_time_interval_flag")
).filter(
    pl.col("zero_time_interval_flag") != 1
).drop(
    "index",
    "time_start",
    "time_end",
    "zero_time_interval_flag"
)[:, 0:1000]

overlap_matrix_vert_sum = overlap_matrix_bars.sum()

overlap_matrix_bars_np = overlap_matrix_bars.to_numpy()
overlap_matrix_vert_sum_np = overlap_matrix_vert_sum.to_numpy()

In [15]:
overlap_matrix_bars[[1,2,3]]

bar_0_overlap,bar_1_overlap,bar_2_overlap,bar_3_overlap,bar_4_overlap,bar_5_overlap,bar_6_overlap,bar_7_overlap,bar_8_overlap,bar_9_overlap,bar_10_overlap,bar_11_overlap,bar_12_overlap,bar_13_overlap,bar_14_overlap,bar_15_overlap,bar_16_overlap,bar_17_overlap,bar_18_overlap,bar_19_overlap,bar_20_overlap,bar_21_overlap,bar_22_overlap,bar_23_overlap,bar_24_overlap,bar_25_overlap,bar_26_overlap,bar_27_overlap,bar_28_overlap,bar_29_overlap,bar_30_overlap,bar_31_overlap,bar_32_overlap,bar_33_overlap,bar_34_overlap,bar_35_overlap,bar_36_overlap,…,bar_963_overlap,bar_964_overlap,bar_965_overlap,bar_966_overlap,bar_967_overlap,bar_968_overlap,bar_969_overlap,bar_970_overlap,bar_971_overlap,bar_972_overlap,bar_973_overlap,bar_974_overlap,bar_975_overlap,bar_976_overlap,bar_977_overlap,bar_978_overlap,bar_979_overlap,bar_980_overlap,bar_981_overlap,bar_982_overlap,bar_983_overlap,bar_984_overlap,bar_985_overlap,bar_986_overlap,bar_987_overlap,bar_988_overlap,bar_989_overlap,bar_990_overlap,bar_991_overlap,bar_992_overlap,bar_993_overlap,bar_994_overlap,bar_995_overlap,bar_996_overlap,bar_997_overlap,bar_998_overlap,bar_999_overlap
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
from tsuro.ensemble.bagging import _generate_sequential_bagging_indices
import time

index_pool = np.arange(overlap_matrix_bars.width)
time0 = time.time()                       
feature_indices, sample_indices = _generate_sequential_bagging_indices(
    overlap_matrix = overlap_matrix_bars_np,
    overlap_matrix_column_sums = overlap_matrix_vert_sum_np,
    sample_index_pool = index_pool,
    n_features = 20,
    max_features = 10,
    max_samples = -1,
    bootstrap_features = False
)
                       
time1 = time.time()
print(f"Time: {time1-time0}")
print(feature_indices)
print(sample_indices)

Time: 5.742009878158569
[ 8 13 18  4 10 16  6  1 15 11]
[263   5 779 625 874 384 767 542 943 713 217 607 849  82 659 271 158 808
 584 445 602 348  65 865 505 283 301 148  73 276 250 441 113 355 912 630
 399 353 381 683 405 658 166 215   5 726 168 362 743 774 326 723 604  12
 723 983 250 796 258 563 945   3 844 775  58 123 642 704  68 340  49 651
 253 571 556 937 278  84 761 184 219 491 372 992 815 864 300 621  11  19
 100 642  16  38 704 499 823 443  87 949 724 958 336  62 729 691 360 171
 332 795 751  12 991 783 910  94 351 113  51 539 783 602 553 508  11 290
 392 733 200 180 597 571 755 434 247 237 613 469 238 164 465 769 641 891
 856 105 418 965  65 821 621 999 597 311 452 996 901 224 317 305 668 489
 108  55 872 766 179 311 112 553 347 534 943 935 972 918 748 737 145 676
 395 401 400 847 736 205 842 815 855 489 400 738 987 111 276  54 621 292
 324 997 955 462 673 670 413 719 997 104   6 854  81  83 436 810 800  33
  60 801 364 940 357 387 102 836 891 917 783 606 821 787 140 538 419

In [20]:
curr_sample_weight = np.ones(1000)

sample_count = np.bincount(sample_indices, minlength = 1000)
print(sample_count)
final_sample_weight = curr_sample_weight*sample_count
final_sample_weight

[1 0 0 1 1 5 3 0 1 0 3 3 3 1 0 1 1 4 1 2 1 0 0 0 2 0 0 0 0 1 0 0 0 1 4 0 0
 1 4 3 1 1 2 1 0 0 1 2 2 1 1 3 1 1 1 3 2 1 1 2 1 0 2 0 0 3 0 0 2 1 1 1 1 3
 0 0 1 2 3 1 0 1 2 2 2 0 0 1 0 1 0 1 1 0 7 0 1 0 1 0 1 0 1 0 1 1 0 0 2 0 0
 1 2 3 2 0 1 2 2 2 0 0 1 1 1 1 1 0 1 2 0 0 0 1 1 0 1 2 1 1 2 1 2 2 1 1 0 2
 2 0 2 1 1 1 2 0 0 0 2 0 0 1 0 0 2 0 1 1 3 2 0 1 1 0 0 0 0 0 0 2 2 2 1 1 1
 1 1 0 0 4 0 1 0 1 1 1 0 0 0 3 1 2 0 1 3 2 1 2 0 0 0 1 0 2 0 1 0 2 1 1 0 1
 0 1 1 2 0 1 0 0 0 0 1 0 1 1 0 2 1 0 0 1 1 1 3 1 1 3 1 1 2 1 0 1 0 0 2 2 2
 0 0 1 0 3 1 0 1 1 0 2 1 1 0 1 1 4 2 0 4 0 0 0 1 2 1 0 1 1 1 0 2 1 1 1 0 1
 0 2 0 1 2 2 0 2 3 1 0 2 1 0 0 3 1 1 0 2 0 3 2 0 0 0 1 0 4 0 2 1 0 1 2 0 2
 1 0 0 3 2 0 2 1 0 1 1 0 1 0 1 1 0 2 2 3 1 0 1 2 1 2 0 2 2 1 1 1 0 0 0 1 0
 2 0 1 1 1 0 0 0 0 1 0 1 1 0 2 0 0 2 1 3 0 1 3 0 0 3 1 0 0 1 3 1 3 1 1 1 0
 0 0 1 0 1 0 3 0 0 0 0 1 3 1 1 0 0 2 0 0 0 1 0 1 2 1 0 2 0 1 0 3 2 2 1 1 1
 0 2 2 5 2 0 0 1 1 3 0 1 1 0 1 0 0 0 1 1 2 1 1 1 0 3 1 0 0 0 1 1 0 0 0 0 0
 3 0 1 1 2 0 0 0 4 1 1 1 

array([1., 0., 0., 1., 1., 5., 3., 0., 1., 0., 3., 3., 3., 1., 0., 1., 1.,
       4., 1., 2., 1., 0., 0., 0., 2., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       4., 0., 0., 1., 4., 3., 1., 1., 2., 1., 0., 0., 1., 2., 2., 1., 1.,
       3., 1., 1., 1., 3., 2., 1., 1., 2., 1., 0., 2., 0., 0., 3., 0., 0.,
       2., 1., 1., 1., 1., 3., 0., 0., 1., 2., 3., 1., 0., 1., 2., 2., 2.,
       0., 0., 1., 0., 1., 0., 1., 1., 0., 7., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 1., 0., 0., 2., 0., 0., 1., 2., 3., 2., 0., 1., 2., 2.,
       2., 0., 0., 1., 1., 1., 1., 1., 0., 1., 2., 0., 0., 0., 1., 1., 0.,
       1., 2., 1., 1., 2., 1., 2., 2., 1., 1., 0., 2., 2., 0., 2., 1., 1.,
       1., 2., 0., 0., 0., 2., 0., 0., 1., 0., 0., 2., 0., 1., 1., 3., 2.,
       0., 1., 1., 0., 0., 0., 0., 0., 0., 2., 2., 2., 1., 1., 1., 1., 1.,
       0., 0., 4., 0., 1., 0., 1., 1., 1., 0., 0., 0., 3., 1., 2., 0., 1.,
       3., 2., 1., 2., 0., 0., 0., 1., 0., 2., 0., 1., 0., 2., 1., 1., 0.,
       1., 0., 1., 1., 2.

In [40]:
from tsuro.sampling.bootstrapping import create_sequential_bootstrap_sample_indices
import time
random_state = np.random.RandomState()
index_pool = np.arange(overlap_matrix_bars.width)
time0 = time.time()
results = [
    create_sequential_bootstrap_sample_indices(
        overlap_matrix = overlap_matrix_bars_np,
        overlap_matrix_column_sums = overlap_matrix_vert_sum_np,
        index_pool = index_pool,
        random_state = random_state
    ) for i in range(12)
]
time1 = time.time()
print(time1-time0)

72.63598728179932


In [44]:
time0 = time.time()
results = Parallel(n_jobs = -1)(
    delayed(create_sequential_bootstrap_sample_indices)(
        overlap_matrix = overlap_matrix_bars_np,
        overlap_matrix_column_sums = overlap_matrix_vert_sum_np,
        index_pool = index_pool
    ) for i in range(12))
time1 = time.time()
print(time1-time0)

30.06018352508545


In [45]:
results

[array([668, 753, 955, 303, 716, 717, 991, 350, 637, 728, 705,  77, 255,
        898,  87, 144, 894, 917, 866, 941, 102, 802, 951, 198, 931, 640,
        702, 141, 319, 322, 460, 118, 731, 814, 161, 635, 887, 635, 552,
        730, 492, 726, 997, 235, 786,  63, 395, 114, 730,  27, 804, 507,
        507, 358,  34, 933, 724, 812, 171, 846, 152, 383, 263,   0,  48,
        795, 459, 905, 580,   4, 965, 607, 300,  68, 734, 809,  85, 392,
        640, 351, 833, 375, 964, 769,  42, 557, 619, 793, 313,  59,  48,
        971,  70, 667, 853, 980, 974, 653,  44, 653, 640, 837, 967, 757,
        617, 340, 757, 630, 865,   5, 761,  43, 596, 670, 584, 674, 400,
        943, 216, 425, 871, 405, 851, 799, 512, 358, 327, 950, 436,  87,
        165, 169, 733, 640, 782, 323, 707, 384, 511, 685, 914, 326, 356,
        679, 797,  12, 675, 385, 525, 611, 673,  51, 970, 228, 590, 471,
         28, 401, 310, 996, 832,  24, 946,  40, 180, 768, 351, 847, 966,
        724, 373, 739, 650, 425, 610, 409, 189, 295

In [50]:
from sklearn.utils.random import sample_without_replacement

def _generate_indices(
    n_population: int,
    n_samples: int,
    bootstrap: bool = False,
    random_state: np.random.RandomState = np.random.RandomState()
) -> np.array:
    """
    Generate random indices
    """
    
    if bootstrap:
        indices = random_state.randint(0, n_population, n_samples)
    else:
        indices = sample_without_replacement(
            n_population, n_samples, random_state = random_state
        )
    return indices

In [59]:
indices = _generate_indices(6,7, bootstrap = True)
indices

array([1, 1, 0, 4, 0, 2, 2])